In [6]:
# %% [markdown]
# # Explore and Merge LAP Coffee Datasets (with PM2.5 and Parks)
# 
# This notebook will:
# 1. Load all GeoPackages
# 2. Inspect structure and date ranges
# 3. Merge daily and static datasets (including PM2.5 and parks)
# 4. Keep only nearest park per address (NOTE: This logic is not implemented in the provided code, but this script will retain all park rows initially if they exist).
# 5. Ensure full daily coverage from 2025-01-01 -> **MODIFIED to cover specific multi-year ranges.**
# 6. Deduplicate rating/user_ratings_total/place_id columns
# 7. Save final CSV for dbt

# %% [markdown]
# ## 1️⃣ Import libraries
import geopandas as gpd
import pandas as pd
from pathlib import Path
from datetime import datetime # Import datetime for date range definition

# %% [markdown]
# ## 2️⃣ Define file paths
data_dir = Path("/Users/tolgasabanoglu/Desktop/github/which-lap-coffee-should-i-visit/data/processed")

gpkg_files = {
    "lst": data_dir / "lap_locations_LST_1km_complete.gpkg",  # PM2.5
    "weather": data_dir / "lap_locations_historical_weather_multi_year.gpkg",
    "ndvi": data_dir / "lap_locations_ndvi_daily_multi_year.gpkg",
    "nightlights": data_dir / "lap_locations_nightlights_monthly_multi_year.gpkg",
    # UPDATED: File name uses 'lap_locations_with_all_bars.gpkg'
    "open_bars": data_dir / "lap_locations_with_open_bars.gpkg", 
    "parks": data_dir / "lap_locations_with_park_counts.gpkg"  # Parks
}

# ----------------------------------------------------------------------
# 🌟 NEW: Define the specific date ranges to cover
# ----------------------------------------------------------------------
TARGET_DATE_RANGES = [
    ("2024-09-01", "2024-11-30"),
    ("2023-09-01", "2023-11-30"),
    # Use the current time for the end date of the 2025 range, as defined in your scripts
    ("2025-09-01", "2025-11-02"), 
]

# %% [markdown]
# ## 3️⃣ Load GeoPackages
gdfs = {}
for name, path in gpkg_files.items():
    print(f"Loading {name} from {path} ...")
    try:
        gdfs[name] = gpd.read_file(path, layer="lap_coffee")
        print(f"{name}: {gdfs[name].shape[0]} rows, columns: {list(gdfs[name].columns)}\n")
    except Exception as e:
        print(f"❌ ERROR: Could not load {name} from {path}. Skipping this file. Error: {e}\n")
        # Ensure the key is removed if loading fails to prevent KeyErrors later
        if name in gpkg_files:
            del gpkg_files[name]
        if name in gdfs:
            del gdfs[name]


# %% [markdown]
# ## 4️⃣ Check date ranges
for name, gdf in gdfs.items():
    print(f"### {name}")
    if "date" in gdf.columns:
        print("Date range:", gdf["date"].min(), "-", gdf["date"].max())
    if "weather_date" in gdf.columns:
        print("Weather date range:", gdf["weather_date"].min(), "-", gdf["weather_date"].max())
    print("\n")

# %% [markdown]
# ## 5️⃣ Normalize date columns
if "weather" in gdfs and "weather_date" in gdfs["weather"].columns:
    gdfs["weather"]["date"] = pd.to_datetime(gdfs["weather"]["weather_date"]).dt.strftime('%Y-%m-%d')

for key in ["lst", "ndvi", "nightlights"]:
    if key in gdfs and "date" in gdfs[key].columns:
        gdfs[key]["date"] = pd.to_datetime(gdfs[key]["date"]).dt.strftime('%Y-%m-%d')


# %% [markdown]
# ## 7️⃣ Merge daily datasets (lst, weather, ndvi, nightlights)
# Using "lst" key for the initial merge (PM2.5).
if "lst" not in gdfs:
    raise FileNotFoundError("Cannot proceed: 'lst' data is missing or failed to load.")
    
daily_merged = gdfs["lst"].copy()

for name in ["weather", "ndvi", "nightlights"]:
    if name in gdfs:
        merge_df = gdfs[name].drop(columns=["geometry", "address"], errors="ignore")
        daily_merged = daily_merged.merge(
            merge_df,
            on=["name", "lat", "lon", "date"],
            how="left",
            suffixes=("", f"_{name}")
        )

# %% [markdown]
# ## 8️⃣ Merge static datasets (parks, open_bars)
# Only includes parks, and open_bars.
for name in ["parks", 'open_bars']:
    if name in gdfs:
        merge_df = gdfs[name].drop(columns=["geometry", "address"], errors="ignore")
        # The parks data might have multiple rows per cafe (for different parks). 
        # This merge keeps all of them, but the final required output needs to address the "nearest park" requirement.
        daily_merged = daily_merged.merge(
            merge_df,
            on=["name", "lat", "lon"],
            how="left",
            suffixes=("", f"_{name}")
        )

# %% [markdown]
# ## 9️⃣ Clean duplicate metadata columns
duplicate_cols = [c for c in daily_merged.columns if any(x in c for x in ["rating_", "user_ratings_total_", "place_id_"])]

if duplicate_cols:
    print("Removing duplicate columns:", duplicate_cols)
    daily_merged = daily_merged.drop(columns=duplicate_cols, errors="ignore")

rename_map = {
    "rating": "cafe_rating",
    "user_ratings_total": "cafe_user_ratings_total",
    "place_id": "cafe_place_id"
}
daily_merged = daily_merged.rename(columns=rename_map)

# %% [markdown]
# ## 🔟 Generate geometry column
daily_merged = gpd.GeoDataFrame(
    daily_merged,
    geometry=gpd.points_from_xy(daily_merged.lon, daily_merged.lat),
    crs="EPSG:4326"
)

# %% [markdown]
# ## 1️⃣1️⃣ Ensure full date coverage per café (Multi-Year Range)
daily_merged["date"] = pd.to_datetime(daily_merged["date"])

# ----------------------------------------------------------------------
# 🌟 FINAL FIX: Generate the exact union of all required dates
# ----------------------------------------------------------------------
all_dates_list = []
for start, end in TARGET_DATE_RANGES:
    all_dates_list.append(pd.date_range(start, end))

# 💡 FIX: Convert each DatetimeIndex to a Series and use pd.concat, 
# which is the most robust way to merge these types of objects across versions.
all_dates = pd.concat([pd.Series(idx) for idx in all_dates_list]).dt.normalize().unique()
# ----------------------------------------------------------------------

print(f"Required date range covers a total of {len(all_dates)} unique days across all specified periods.")

# Identify unique cafes and static columns
cafes_static_cols = ['name', 'lat', 'lon', 'address', 'cafe_rating', 'cafe_user_ratings_total', 'cafe_place_id']
# Also include static park/bar columns that are now duplicated across daily rows
static_cols_to_keep = [
    col for col in daily_merged.columns 
    if col not in ['date', 'lst_celsius_1km', 'temp_max', 'temp_min', 'precip_mm', 'ndvi', 'nightlight', 'geometry'] and col not in cafes_static_cols
]


cafes = daily_merged[cafes_static_cols + static_cols_to_keep].drop_duplicates(subset=['name', 'lat', 'lon']).reset_index(drop=True)


# Create the Cartesian product (full index)
full_index = pd.MultiIndex.from_product([cafes.index, all_dates], names=["cafe_idx", "date"])
full_df = pd.DataFrame(index=full_index).reset_index()

# Merge static cafe metadata back into the full date frame
full_df = full_df.merge(cafes.reset_index().rename(columns={'index': 'cafe_idx_merge'}), 
                        left_on="cafe_idx", 
                        right_on="cafe_idx_merge", 
                        how="left")
full_df = full_df.drop(columns=["cafe_idx", "cafe_idx_merge"])

# Ensure date is datetime for final merge
full_df["date"] = pd.to_datetime(full_df["date"])

# Merge the original, sparse merged data onto the complete date frame
daily_merged = full_df.merge(
    # Drop static columns from the merged data before the final merge to avoid redundant columns/data
    daily_merged.drop(columns=static_cols_to_keep + ['cafe_rating', 'cafe_user_ratings_total', 'cafe_place_id', 'address'], errors='ignore'),
    on=["name", "lat", "lon", "date"],
    how="left"
)

print(f"✅ Expanded dataset covers {len(all_dates)} days × {len(cafes)} cafes = {len(all_dates) * len(cafes):,} rows")

# %% [markdown]
# ## 1️⃣2️⃣ Convert date to string (for dbt / CSV)
daily_merged["date"] = daily_merged["date"].dt.strftime('%Y-%m-%d')

# %% [markdown]
# ## 1️⃣3️⃣ Save final merged dataset
output_csv = data_dir / "lap_locations_final_merged.csv"
daily_merged.to_csv(output_csv, index=False)
print(f"✅ Merged dataset saved: {output_csv}")

# %% [markdown]
# ## 1️⃣4️⃣ Quick summary
print("Columns:", list(daily_merged.columns))
print("Number of rows:", daily_merged.shape[0])
print("Sample rows:\n", daily_merged.head())

Loading lst from /Users/tolgasabanoglu/Desktop/github/which-lap-coffee-should-i-visit/data/processed/lap_locations_LST_1km_complete.gpkg ...
lst: 3304 rows, columns: ['name', 'address', 'lat', 'lon', 'date', 'lst_celsius_1km', 'geometry']

Loading weather from /Users/tolgasabanoglu/Desktop/github/which-lap-coffee-should-i-visit/data/processed/lap_locations_historical_weather_multi_year.gpkg ...
weather: 3920 rows, columns: ['weather_date', 'temp_max', 'temp_min', 'precip_mm', 'name', 'address', 'lat', 'lon', 'rating', 'user_ratings_total', 'season', 'geometry']

Loading ndvi from /Users/tolgasabanoglu/Desktop/github/which-lap-coffee-should-i-visit/data/processed/lap_locations_ndvi_daily_multi_year.gpkg ...
ndvi: 3920 rows, columns: ['name', 'address', 'lat', 'lon', 'date', 'ndvi', 'geometry']

Loading nightlights from /Users/tolgasabanoglu/Desktop/github/which-lap-coffee-should-i-visit/data/processed/lap_locations_nightlights_monthly_multi_year.gpkg ...
nightlights: 3920 rows, columns: